In [6]:
"""
Mounting the Google drive
"""
from google.colab import drive
drive.mount('/content/drive')

root_path = "/content/drive/MyDrive/Colab Notebooks/COEN240_TA/data"
train_path = root_path + "/train"
grade_path = root_path + "/grade"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import joblib
import os
pca = joblib.load(os.path.join(train_path, 'pca_model.sav'))
lda = joblib.load(os.path.join(train_path, 'lda_model.sav'))

In [10]:
import pickle
import os

ensemble1_path = os.path.join(train_path, 'ensemble_model1.pkl')
ensemble2_path = os.path.join(train_path, 'ensemble_model2.pkl')

# Load the pickle file
# Load the pickle file
# with open(ensemble1_path, 'rb') as file:
#     ensemble1 = pickle.load(file)

with open(ensemble2_path, 'rb') as file:
    ensemble2 = pickle.load(file)

In [11]:
def face_box(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces_detected = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces_detected:
      image_face = image[y:y+h, x:x+w]
      resized_image_face = cv2.resize(image_face, (200, 200))
      return resized_image_face

In [12]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

labels = []
d = {}
for folder in os.listdir(train_path):
  if os.path.isfile(os.path.join(train_path, folder)):
    continue
  if folder.endswith("npy"):
    continue
  label = folder[:-1]
  labels.append(label)

encoded_labels = label_encoder.fit_transform(labels)
for label, encoded_label in zip(labels, encoded_labels):
    d[encoded_label] = label
labels = encoded_labels

In [43]:
import os
import cv2
import numpy as np
import pickle
from google.colab.patches import cv2_imshow
import pandas as pd
from os.path import isfile, join

# Path to the folder containing test images
test_images_folder = grade_path

# Get the list of test image filenames
test_image_files = os.listdir(test_images_folder)

# Initialize lists to store filenames and predictions
filenames = []
predictions = []

# Initialize list to store preprocessed face images
preprocessed_images = []
image_files = []
final_predictions = {}
# Iterate over each test image
for image_file in test_image_files:
    if not image_file.endswith("jpeg"):
      continue
    if not isfile(os.path.join(grade_path, image_file)):
      continue
    image_files.append(image_file)
    # Load the test image
    image_path = os.path.join(test_images_folder, image_file)
    image = cv2.imread(image_path)
    print(image_file)

    face_cropped_image = face_box(image)
    if face_cropped_image is None:
      face_cropped_image = cv2.resize(image, (200, 200))
    image_grey = cv2.cvtColor(face_cropped_image, cv2.COLOR_BGR2GRAY)
    flattened_image = [image_grey.flatten()]
    image_pca = pca.transform(flattened_image)
    image_lda = lda.transform(image_pca)
    pred = ensemble2.predict(image_lda)
    predictions.append(d[pred[0]])
    print(d[pred[0]])
    final_predictions[image_file] = d[pred[0]]

# Create a DataFrame with filenames and predictions
data = {'filename': image_files, 'predictions': predictions}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file_path = os.path.join(grade_path, 'predictions.csv')
df.to_csv(csv_file_path, index=False)

print("Predictions saved to:", csv_file_path)

0123_2.jpeg
vanderlindenilona
0123_1.jpeg
wukaiyue
0123_3.jpeg
zhangyuanzhen
Predictions saved to: /content/drive/MyDrive/Colab Notebooks/COEN240_TA/data/grade/predictions.csv


In [44]:
def calculate_accuracy(ground_truths, predictions):
    if len(ground_truths) != len(predictions):
        raise ValueError("The number of predictions does not match the number of ground truths.")
    correct_predictions = 0
    for sample_id, ground_truth in ground_truths.items():
        if predictions.get(sample_id) == ground_truth:
            correct_predictions += 1
    return correct_predictions / len(ground_truths)

def grade_predictions(predictions, grade_path=grade_path):
    df = pd.read_csv(grade_path + "/solution.csv")
    ground_truths = dict(zip(df['filename'], df['ground_truth']))
    accuracy = calculate_accuracy(ground_truths, predictions)
    return accuracy

accuracy = grade_predictions(final_predictions)
print(f"Prediction Accuracy: {accuracy * 100}%")

Prediction Accuracy: 100.0%
